## Molecular Dynamics

Run molecular dynamics (MD) simulations using [GPUMD](https://github.com/brucefan1983/GPUMD).

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Creating bulk atoms.

In [ ]:
from wizard.atoms import SymbolInfo, Morph
atoms = SymbolInfo('MoTaVW', 'bcc', 3).create_bulk_atoms((20, 20, 20))

Running molecular dynamics simulations.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1', 
          'ensemble npt_scr 300 300 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_restart 30000', 
          'dump_exyz 10000',
          'run 30000']
Morph(atoms).gpumd('relax', run_in)

Deforming the simulation box.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1',
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'run 30000', 
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'deform 0.00001 0 0 1', 
          'dump_thermo 1000', 
          'dump_exyz 1000', 
          'dump_restart 10000',
          'run 1000000']
Morph(atoms).gpumd('deform', run_in)

Simulating the process of a crystallization.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 2000', 
          'time_step 1', 
          'ensemble npt_scr 2000 2000 200 0 500 2000', 
          'dump_thermo 1000', 
          'dump_exyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 2000 5000 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 100000', 
          'dump_restart 10000', 
          'run 10000000',
          'ensemble npt_scr 4500 4500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 4500 1500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_exyz 100000', 
          'dump_restart 10000', 
          'run 10000000']
Morph(atoms).gpumd('crystallization', run_in)

Calculating the Melting point using two-phase coexistence method.

In [ ]:
from wizard.io import read_restart

group = []
for atom in atoms:
    if atom.position[2] < atoms.cell[2, 2] / 2:
        group.append(0)
    else:
        group.append(1)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt', 
            'velocity 3000', 
            'time_step 1', 
            'ensemble npt_ber 3000 3000 200 0 500 2000', 
            'dump_exyz 10000', 
            'dump_thermo 1000',
            'run 30000',
            'ensemble heat_lan 3500 200 500 0 1',
            'dump_exyz 10000',
            'dump_thermo 1000',
            'dump_restart 10000',
            'run 1000000']

Morph(atoms).gpumd('melting_point/relax', run_in_1)

for Tm in range(3400, 3701, 100):
    atoms = read_restart('melting_point/relax/restart.xyz')
    run_in = ['potential ../../nep.txt', 
             f'velocity {Tm}', 
              'time_step 1', 
             f'ensemble npt_ber {Tm} {Tm} 200 0 500 2000', 
              'dump_exyz 10000', 
              'dump_thermo 1000',
              'run 30000']
    Morph(atoms).gpumd(f'melting_point/{Tm}', run_in)

Simulating the radiation damage.

In [ ]:
from wizard.io import read_restart
import numpy as np

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 30000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 70000']

pka_energy = 10 #eV
direction = np.array([1, 3, 5]) 
index = 100

Morph(atoms).gpumd('radiation/relax', run_in_1)
atoms = read_restart('radiation/relax/restart.xyz')
Morph(atoms).set_pka(pka_energy, direction, index)
Morph(atoms).gpumd('radiation/cascade', run_in_2)

Simulating the overlapping cascades.

In [ ]:
from wizard.io import read_restart
import numpy as np
import random

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 10000']

run_in_3 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 40000']

pka_energy = 10 #eV
cascade_times = 2000
directions = [np.array([np.sin(np.random.uniform(0, np.pi)) * np.cos(np.random.uniform(0, 2 * np.pi)),
                        np.sin(np.random.uniform(0, np.pi)) * np.sin(np.random.uniform(0, 2 * np.pi)),
                        np.cos(np.random.uniform(0, np.pi))]) for _ in range(cascade_times)]

indexs = [random.randint(0, len(atoms) - 1) for _ in range(cascade_times)]

## First time
direction = directions[0]
index = indexs[0]

center = atoms.cell.diagonal() / 2
diff = center - atoms[index].position
for atom in atoms:
    atom.position += diff

for atom in atoms:
    atom.position %= atoms.cell.diagonal()

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

Morph(atoms).gpumd('radiation0/relax', run_in_1)
atoms = read_restart('radiation0/relax/restart.xyz')
Morph(atoms).set_pka(pka_energy, direction, index)
Morph(atoms).gpumd('radiation0/cascade', run_in_3)

## Loops
for i in range(1, cascade_times):
    direction = directions[i]
    index = indexs[i]
    atoms = read_restart(f'radiation{i-1}/cascade/restart.xyz')

    center = atoms.cell.diagonal() / 2
    diff = center - atoms[index].position
    for atom in atoms:
        atom.position += diff

    for atom in atoms:
        atom.position %= atoms.cell.diagonal()

    group = []
    thickness = 3.185 * 3
    for atom in atoms:
        if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
            group.append(0)
        elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
            group.append(1)
        else:
            group.append(2)
    atoms.info['group'] = group

    Morph(atoms).gpumd(f'radiation{i}/relax', run_in_2)
    atoms = read_restart(f'radiation{i}/relax/restart.xyz')
    Morph(atoms).set_pka(pka_energy, direction, index)
    Morph(atoms).gpumd(f'radiation{i}/cascade', run_in_3)

Calculate the threshold displacement energy surface

In [ ]:
import numpy as np

group = []
thickness = 3.185
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
        
atoms.info['group'] = group

theta_range = np.arange(0, 46, 5) 
phi_range = np.arange(0, 46, 5) 

directions = []

for theta in theta_range:
    for phi in phi_range:
        theta_rad = np.deg2rad(theta)
        phi_rad = np.deg2rad(phi)
        x = np.sin(theta_rad) * np.cos(phi_rad)
        y = np.sin(theta_rad) * np.sin(phi_rad)
        z = np.cos(theta_rad)
        directions.append(np.array([x, y, z]))

run_in = ['potential ../../nep.txt', 
          'velocity 1', 
          'time_step 0', 
          'ensemble nve',
          'dump_exyz 1', 
          'run 1',
          'time_step 1 0.015', 
          'ensemble heat_nhc 10 200 0 0 1',
          'electron_stop ../../electron_stopping_fit.txt',
          'dump_exyz 500 1 1',
          'run 5000']

atoms.info['velocities'] = np.zeros((len(atoms), 3))

for direction in directions:
    for pka_energy in range(10, 101, 10):
        index = 8420
        Morph(atoms).set_pka(pka_energy, direction, index)
        Morph(atoms).gpumd(f'{direction}/{pka_energy}', run_in)